In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt 
import seaborn
import statsmodels.api as sm
from scipy.stats import zscore
from statsmodels import regression
from statsmodels.tsa.stattools import coint

In [248]:
class Pair_trading():
    def __init__(self, data_var,data_preco, par, valor_inicial):
        self.financeiro = valor_inicial
        self.posicionado = 0 #comeca sem nenhuma operação aberta
        #guarda tickers das acoes
        self.t1 = par[0] 
        self.t2 = par[1]
        
        self.volume_operacao = 0
        
        self.dados_var = data_var
        self.dados_preco = data_preco

    def calcula_zscore_spread(self,s1,s2):
        s1 = sm.add_constant(s1)
        results = sm.OLS(s2,s1).fit()
        s1 = s1[self.t1]
        b = results.params[self.t1]
        spread = s2 - (b*s1)
        zs = zscore(spread)[-1]
        return zs
        
    def condicoes_operar(self,zs, gatilho_high, preco_s1, preco_s2):
        if zs >= gatilho_high and self.posicionado == 0:
            self.razao = preco_s1/preco_s2
            
            self.posicionado = 1 #entrou em uma operacao
            
            self.financeiro += preco_s1
            self.financeiro -= preco_s2*self.razao

        elif zs <= 0.08 and self.posicionado == 1:
            self.posicionado = 0
            
            self.financeiro -= preco_s1
            self.financeiro += preco_s2*self.razao
            
            self.razao = 0
        
        
    def opera(self):
        period = 45
        c = 0
        s1 = self.dados_var[self.t1]
        s2 = self.dados_var[self.t2]
        for index in range(period, len(self.dados_var)):
            c += 1
            print(c)

            s1 = s1[c:index]
            print(s1)

            s2 = s2[c:index]
            print(len(s1), len(s2))
            preco_s1 = self.dados_preco[self.t1][self.dados_var.index == index].values[0]
            preco_s2 = self.dados_preco[self.t2][self.dados_var.index == index].values[0]
#             print(f'p1: {preco_s1:.2f} p2: {preco_s2:.2f}')
            zs = self.calcula_zscore_spread(s1=s1,s2=s2)
            self.condicoes_operar(zs=zs, gatilho_high = 1.1, preco_s1=preco_s1, preco_s2=preco_s2)

        

In [249]:
data = pd.read_excel('/home/daniel/Desktop/Projetos/mql_python/Relatório 1/dados_relatorio.xlsx')[1:]

data_var = data.drop('time',axis=1).pct_change()[1:]
data_preco = data[1:]

In [250]:
tickers = [['close ITSA4','close ITUB4']]#, ['close BBAS3','close BBSE3']]

In [251]:
dic = {}
for i in tickers:
    a = (i[0]+' x '+i[1])
    print(a)
    dic[a] = Pair_trading(data_var=data_var, data_preco=data_preco, par=i, valor_inicial=1000)
    dic[a].opera()
    print('\n\n\n\n\n')


close ITSA4 x close ITUB4
1
3     0.005249
4     0.005222
5     0.015584
6     0.000000
7    -0.002558
8    -0.005128
9     0.000000
10    0.002577
11    0.025707
12    0.005013
13    0.000000
14   -0.012469
15   -0.017677
16    0.010283
17   -0.012723
18   -0.010309
19   -0.007812
20    0.018373
21    0.002577
22    0.002571
23    0.002564
24    0.010230
25    0.020253
26   -0.004963
27   -0.014963
28    0.025316
29   -0.007407
30   -0.012438
31    0.000000
32    0.005038
33   -0.005013
34   -0.007557
35   -0.005076
36   -0.007653
37    0.020566
38    0.005038
39    0.032581
40    0.014563
41   -0.011962
42   -0.029056
43    0.017456
44    0.024510
45    0.009569
46   -0.014218
Name: close ITSA4, dtype: float64
44 44
2
5     0.015584
6     0.000000
7    -0.002558
8    -0.005128
9     0.000000
10    0.002577
11    0.025707
12    0.005013
13    0.000000
14   -0.012469
15   -0.017677
16    0.010283
17   -0.012723
18   -0.010309
19   -0.007812
20    0.018373
21    0.002577
22    0.002571


ValueError: zero-size array to reduction operation maximum which has no identity

In [243]:
dic['close ITSA4 x close ITUB4'].financeiro

1000